In [1]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [2]:
df4=pd.read_csv('Cap/Empath_variables.csv', delimiter = ',',encoding='utf-8')
df5=df4.copy()
df4.drop(df4.columns.difference(['Username','Subreddit','Depressed']), 1, inplace=True)
df4.drop_duplicates(keep='first',inplace=True) 
df4.reset_index(drop=True)
df4 = df4.astype(str)
df6=df4.groupby(['Username','Depressed'])['Subreddit'].apply(lambda x:','.join(x)).reset_index(name ='Subreddits')

In [3]:
df6.head()

,Username,Depressed,Subreddits
0,#NAME?,0,"baseball,TalesFromtheLoopRPG,whowouldwin,video..."
1,-Chely-,1,"Food_Pantry,JUSTNOMIL,depression,Assistance"
2,-Darth_Revan-,1,depression
3,-ExtremeMusic-,0,"whowouldwin,tf2"
4,-GrapeGrass-,0,"whowouldwin,CharacterRant,unpopularopinion,per..."


In [4]:
df6['Subreddits'] = df6['Subreddits'].str.replace('depression,', '')

In [5]:
df6.head()

,Username,Depressed,Subreddits
0,#NAME?,0,"baseball,TalesFromtheLoopRPG,whowouldwin,video..."
1,-Chely-,1,"Food_Pantry,JUSTNOMIL,Assistance"
2,-Darth_Revan-,1,depression
3,-ExtremeMusic-,0,"whowouldwin,tf2"
4,-GrapeGrass-,0,"whowouldwin,CharacterRant,unpopularopinion,per..."


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df6.Subreddits)
X_train_counts.shape

(4479, 8586)

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts).toarray()
X_train_tfidf.shape

(4479, 8586)

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
scores = cross_val_score(clf,X_train_tfidf, df6.Depressed, cv=10, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.74 (+/- 0.04)


In [9]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
scores = cross_val_score(clf,X_train_tfidf, df6.Depressed, cv=10, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.90 (+/- 0.02)


In [10]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=5, random_state=42)
scores = cross_val_score(clf,X_train_tfidf, df6.Depressed, cv=10, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

C:\Users\asus\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\asus\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\asus\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier

Accuracy: 0.91 (+/- 0.03)


In [11]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
scores = cross_val_score(clf,X_train_tfidf, df6.Depressed, cv=10, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.62 (+/- 0.06)
